# i.MXRT105x flashloader example
This is example how to create flashloader for i.MXRT105x device. The aim of this Jupyter notebook is show how to easily create HAB container with flashloader application from NXP SDK to be loaded with serial downloader mode.
The process could be split to several steps.

Note:
-----
This example could be rewritten to any other NXP device with HAB container in case of need (like i.MXRT117x, etc)

## 1. Compile the flashloader application
As a first step is needed to compile flashloader application itself from NXP SDK. 
    - Download the NXP MCUXpresso SDK from NXP web site (https://mcuxpresso.nxp.com/)
    - Open the flashloader application in your favorite IDE. (Regular path to application: {SDK_root}\boards\evkmimxrt1050\bootloader_examples\flashloader\cm4)
    - Compile the flashloader and generate the binary output (In our example we use precompiled flashloader_exe.bin)

## 2. Prepare the HAB configuration file
Run this code to initialize members for this example...

In [1]:
%run ../../init_notebook.ipynb

import os
import pprint

pp = pprint.PrettyPrinter(indent=4)

WORKSPACE = "workspace/"  # change this to path to your workspace
VERBOSITY = (
    "-v"  # verbosity of commands, might be -v or -vv for debug or blank for no additional info
)

TEMPLATE_FILE = WORKSPACE + "hab_template.yaml"  # HAB configuration template file
FLSHLDR_EXE_PATH = (
    "inputs/flashloader_exe.bin"  # Input compiled flashloader executable in binary format
)
HAB_CONFIG = "inputs/flashloader_cfg.yaml"  # Prepared HAB configuration file
FLASHLOADER = WORKSPACE + "flashloader.bin"  # Final Bootable flashloader

env: JUPYTER_SPSDK=1
Created `%!` as an alias for `%execute`.


To get all supported sub commands of HAB support call help.

In [2]:
%! nxpimage hab

nxpimage hab 
Usage: nxpimage hab [OPTIONS] COMMAND [ARGS]...

  Group of sub-commands related to HAB container.

Options:
  --help  Show this message and exit.

Commands:
  convert       Convert BD Configuration to YAML.
  export        Generate HAB container from configuration.
  get-template  Create template of configuration in YAML format.
  parse         Parse HAB container into individual segments.


### As a good starting point it could be creation of HAB configuration YAML file template

In [3]:
# Create template HAB configuration file using nxpimage
%! nxpimage $VERBOSITY hab get-template -o $TEMPLATE_FILE --force

assert os.path.exists(TEMPLATE_FILE)

nxpimage -v hab get-template -o workspace/hab_template.yaml --force 
Creating workspace/hab_template.yaml template file.


### Modify the template to configuration file used to create AHAB flashloader as a serial downloader mode

1. Change the Input Image file to precompiled file\
   ![Change the Input Image file](IMG/6_image-7.png)
2. Change the flags to unsigned HAB output\
   ![Flags to unsigned image](IMG/6_image-2.png)
3. Change the start address to proper value. Linker interrupt table value minus 0x400. In our case 0x20002000 - 0x400 = 0x20001c00\
   ![Start address change](IMG/6_image.png)
   For example source linker value from the IAR icf file:\
   ![Interrupt value of start of executable](IMG/6_image-3.png)
4. Set IVT offset to zero and initial load size 0x400:\
   ![IVT offset to 0, initial load size to 0x400](IMG/6_image-9.png)
5. Set the proper Entry point address. The valid address could be find in compiled binary at vector 1 in interrupt vector table of application. The value is in little endian format. Here is the example for our case: 0x20017379\

    ![Binary application to get entry point](IMG/6_image-4.png)
    
    The configuration file value:\
    ![Entry point definition](IMG/6_image-5.png)
6. Remove optional 'signatureTimestamp', 'DCDFilePath' and 'sections' what is not needed in our case

## 3. Export the HAB container
To export HAB container is designed SPSDK tool in 'nxpimage' under 'hab' sub commands group.


In [4]:
# Export HAB container using nxpimage
%! nxpimage $VERBOSITY hab export -c $HAB_CONFIG -o $FLASHLOADER

assert os.path.exists(HAB_CONFIG)

nxpimage -v hab export -c inputs/flashloader_cfg.yaml -o workspace/flashloader.bin 
Success. (HAB container: workspace/flashloader.bin created.)


## 4. Generated flashloader file load to i.MXRT105x
As a final step is load flashloader into MCU and run. The easiest way is get 'sdphost' application and 'write-file'/'jump-address' command.

Chip must be in serial downloader mode!

Here is two examples how to do it over Serial line:
- `sdphost -p com1 write-file 0x20001c00 workspace\flashloader.bin`
- `sdphost -p com1 jump-address 0x20001c00`